#### DenseNet

In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.nn import functional as F
import torch.nn as nn
from torch.autograd import Variable
from tqdm import tqdm
from collections import OrderedDict
from uuid import uuid4

In [152]:
def make_initial_block_config(n_init_features, bias=False):
    prefix = "initial_block"
    block_config = {}
    block_config[f"{prefix}.conv_0"] = {
        "name": "Conv2d",
        "parameters": {
          "in_channels": 3,
          "out_channels": n_init_features,
          "kernel_size": 5,
          "stride": 1,
          "bias": bias
        }
      }
    block_config[f"{prefix}.batch_norm_0"] = {
        "name": "BatchNorm2d",
        "parameters": {
          "num_features": n_init_features
        }
      }
    block_config[f"{prefix}.non_linearity_0"] = {
        "name": "LeakyReLU",
        "parameters": {
          "inplace": True
        }
    }
    return block_config

In [153]:
def make_dense_block_config(block_i, n_layers, n_input_features, growth_rate, bottleneck_size, dropout_rate, bias=False):
    block_config = OrderedDict()
    for layer_i in range(n_layers):
        layer_config = make_dense_layer_config(block_i, layer_i, n_input_features + layer_i * growth_rate, growth_rate,
                           bottleneck_size, dropout_rate, bias)
        block_config.update(**layer_config)
    return block_config

In [154]:
def make_dense_layer_config(block_i, layer_i, n_input_features, growth_rate, bottleneck_size, dropout_rate, bias=False):
    layer_config = OrderedDict()
    prefix = f"denseblock_{block_i}.layer_{layer_i}"
    layer_config[f"{prefix}.batch_norm_0"] = {
        "name": "BatchNorm2d",
        "parameters": {
          "num_features": n_input_features
        }
      }
    layer_config[f"{prefix}.non_linearity_0"] = {
        "name": "LeakyReLU",
        "parameters": {
          "inplace": True
        }
      }
    layer_config[f"{prefix}.conv_0"] = {
        "name": "Conv2d",
        "parameters": {
          "in_channels": n_input_features,
          "out_channels": growth_rate * bottleneck_size,
          "kernel_size": 1,
          "stride": 1,
          "bias": bias
        }
      }
    layer_config[f"{prefix}.dropout_0"] = {
            "name": "Dropout",
            "parameters": {
                "p": dropout_rate
            }
          }
    layer_config[f"{prefix}.batch_norm_1"] = {
        "name": "BatchNorm2d",
        "parameters": {
          "num_features": growth_rate * bottleneck_size
        }
      }
    layer_config[f"{prefix}.non_linearity_1"] = {
        "name": "LeakyReLU",
        "parameters": {
          "inplace": True
        }
      }
    layer_config[f"{prefix}.conv_1"] = {
        "name": "Conv2d",
        "parameters": {
          "in_channels": growth_rate * bottleneck_size,
          "out_channels": growth_rate,
          "kernel_size": 3,
          "stride": 1,
          "padding": 1,
          "bias": bias
        }
      }
    layer_config[f"{prefix}.dropout_1"] = {
        "name": "Dropout",
        "parameters": {
            "p": dropout_rate
        }
      }
    return layer_config 

In [155]:
def make_transition_layer_config(block_i, n_input_features, n_output_features, bias=False):
    prefix = f"transition_{block_i}"
    layer_config = OrderedDict()
    layer_config[f"{prefix}.batch_norm_0"] = {
        "name": "BatchNorm2d",
        "parameters": {
          "num_features": n_input_features
        }
      }
    layer_config[f"{prefix}.non_linearity_0"] = {
        "name": "LeakyReLU",
        "parameters": {
          "inplace": True
        }
      }
    layer_config[f"{prefix}.conv_0"] = {
        "name": "Conv2d",
        "parameters": {
          "in_channels": n_input_features,
          "out_channels": n_output_features,
          "kernel_size": 1,
          "stride": 1,
          "padding": 1,
          "bias": bias
        }
    }
    layer_config[f"{prefix}.pooling_0"] = {
        "name": "AvgPool2d",
        "parameters": {
           "kernel_size": 2,
            "stride": 2
        }
    }
    return layer_config

In [156]:
config = {
    "model": {
        "name": "DenseNet",
        "config": {
          "n_initial_features": 32,
          "block_config": [6, 16, 24, 16],
          "growth_rate": 16, 
          "bottleneck_size": 4
        }
    }, 
    "optimizer": {
    "name": "ADAM",
    "parameters": {
      "weight_decay": 0.0001,
      "lr": 0.00005
    }
  },
  "loss_function": {
    "name": "BCELoss"
  },
  "output_transformation": {
    "name": "sigmoid"
  },
  "trainer_delegate": {
    "name": "DenseNet"
  },
  "data_source_delegate": {
    "name": "DenseNet",
    "parameters": {
      "batch_size": 32,
      "training_data_path": "/home/kevin/workspace/Kaggle/Iceberg-Classifier-Challenge/src/data/train.json",
      "testing_data_path": "/home/kevin/workspace/Kaggle/Iceberg-Classifier-Challenge/src/data/test.json"
    }
  },
  "saver_delegate": {
    "name": "DenseNet"
  },
  "n_epochs": 35
}

In [151]:
n_initial_features = 32
layers_per_block = [4, 8, 12, 8]
growth_rate = 8
bottleneck_size = 2

In [157]:
n_features = n_initial_features
model_config = {}
initial_block = make_initial_block_config(n_features)
model_config.update(**initial_block)
for block_i, n_layers in enumerate(layers_per_block):
    block_config = make_dense_block_config(block_i, n_layers, n_features, growth_rate, bottleneck_size, dropout_rate)
    model_config.update(**block_config)
    n_features = n_features + growth_rate * n_layers
    if block_i != len(block_config) - 1:
        trans = make_transition_layer_config(block_i, n_input_features=n_features, n_output_features=n_features // 2)
        n_features = n_features // 2
        model_config.update(**trans)

In [158]:
config["model"]["config"].update(**model_config)
config.update({"id": str(uuid4())})

In [159]:
with open("/home/kevin/workspace/Kaggle/Iceberg-Classifier-Challenge/study_configs/densenet.json", "w") as fp:
    json.dump(config, fp, indent=2)

In [160]:
config

{'data_source_delegate': {'name': 'DenseNet',
  'parameters': {'batch_size': 32,
   'testing_data_path': '/home/kevin/workspace/Kaggle/Iceberg-Classifier-Challenge/src/data/test.json',
   'training_data_path': '/home/kevin/workspace/Kaggle/Iceberg-Classifier-Challenge/src/data/train.json'}},
 'id': 'b3ab8115-6fc1-47b9-ae3e-b2caf85500c6',
 'loss_function': {'name': 'BCELoss'},
 'model': {'config': {'block_config': [6, 16, 24, 16],
   'bottleneck_size': 4,
   'denseblock_0.layer_0.batch_norm_0': {'name': 'BatchNorm2d',
    'parameters': {'num_features': 32}},
   'denseblock_0.layer_0.batch_norm_1': {'name': 'BatchNorm2d',
    'parameters': {'num_features': 64}},
   'denseblock_0.layer_0.conv_0': {'name': 'Conv2d',
    'parameters': {'bias': False,
     'in_channels': 32,
     'kernel_size': 1,
     'out_channels': 64,
     'stride': 1}},
   'denseblock_0.layer_0.conv_1': {'name': 'Conv2d',
    'parameters': {'bias': False,
     'in_channels': 64,
     'kernel_size': 3,
     'out_channels